In [2]:
!pip install snowflake['ml']

     |████████████████████████████████| 450kB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 37.8MB/s eta 0:00:01
     |████████████████████████████████| 235kB 41.1MB/s eta 0:00:01
     |████████████████████████████████| 409kB 48.9MB/s eta 0:00:01
     |████████████████████████████████| 122kB 48.0MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 34.8MB/s eta 0:00:01
     |████████████████████████████████| 737kB 31.8MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 76.9MB/s eta 0:00:01   |█▏                              | 7.6MB 39.1MB/s eta 0:00:05
     |████████████████████████████████| 12.2MB 27.3MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 43.5MB/s eta 0:00:01
     |████████████████████████████████| 133kB 51.2MB/s eta 0:00:01
     |████████████████████████████████| 41.1MB 51.1MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 42.1MB/s eta 0:00:01
     |████████████████████████████████| 174kB 49.3MB/s eta 0:

     |████████████████████████████████| 2.1MB 43.6MB/s eta 0:00:01
     |████████████████████████████████| 71kB 36.1MB/s eta 0:00:01
     |████████████████████████████████| 870kB 39.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 45.1MB/s eta 0:00:01
     |████████████████████████████████| 143kB 27.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 32.9MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 28.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 44.5MB/s eta 0:00:01
     |████████████████████████████████| 163kB 45.5MB/s eta 0:00:01
     |████████████████████████████████| 450kB 42.6MB/s eta 0:00:01
     |████████████████████████████████| 61kB 39.7MB/s eta 0:00:01
     |████████████████████████████████| 112kB 50.5MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 46.8MB/s eta 0:00:01
     |████████████████████████████████| 307kB 34.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 46.2MB/s eta 0:00:01


     |████████████████████████████████| 12.2MB 27.1MB/s eta 0:00:01
     |████████████████████████████████| 92kB 21.0MB/s eta 0:00:01
ERROR: refractio 2.1.5.5 has requirement pandas==2.0.0, but you'll have pandas 1.5.3 which is incompatible.
ERROR: mlflow 2.10.0 has requirement pyarrow<16,>=4.0.0, but you'll have pyarrow 16.1.0 which is incompatible.
ERROR: mlflow 2.10.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: snowflake-snowpark-python 1.17.0 has requirement cloudpickle!=2.1.0,!=2.2.0,<=2.2.1,>=1.6.0; python_version < "3.11", but you'll have cloudpickle 3.0.0 which is incompatible.
ERROR: snowflake-connector-python 3.10.1 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.2.1 which is incompatible.
ERROR: openapi-spec-validator 0.7.1 has requirement jsonschema<5.0.0,>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StructType, StructField, DoubleType, StringType
import snowflake.snowpark.functions as F

In [2]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": "Enlightme#2024",
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
#     "authenticator": "externalbrowser", # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
} 

In [3]:
# Make a Snowpark Connection

################################################################################################################
#  You can also use the SnowSQL Client to configure your connection params:
#  https://docs.snowflake.com/en/user-guide/snowsql-install-config.html
#
#  >>> from snowflake.ml.utils import connection_params
#  >>> session = Session.builder.configs(connection_params.SnowflakeLoginOptions()
#  >>> ).create()   
#
#  NOTE: If you have named connection params then specify the connection name
#  Example:
#  
#  >>> session = Session.builder.configs(
#  >>> connection_params.SnowflakeLoginOptions(connection_name='connections.snowml')
#  >>> ).create()
#
#################################################################################################################

# Edit the connection.json before creating the session object below
# Create Snowflake Session object
# connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : ADITYASINGH
Role                        : "ADITYASINGH"
Database                    : "FIRST_DB"
Schema                      : "PUBLIC"
Warehouse                   : "FOSFOR_INSIGHT_WH"
Snowflake version           : 8.20.0
Snowpark for Python version : 1.17.0


In [4]:
import numpy as np
import pandas as pd
import random
import string

from sklearn.datasets import make_regression
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.snowpark import Session

# Create a session with your preferred method
# session =

NUMERICAL_COLS = ["X1", "X2", "X3"]
CATEGORICAL_COLS = ["C1", "C2", "C3"]
FEATURE_COLS = NUMERICAL_COLS + CATEGORICAL_COLS
CATEGORICAL_OUTPUT_COLS = ["C1_OUT", "C2_OUT", "C3_OUT"]
FEATURE_OUTPUT_COLS = ["X1_FEAT_OUT", "X2_FEAT_OUT", "X3_FEAT_OUT", "C1_FEAT_OUT", "C2_FEAT_OUT", "C3_FEAT_OUT"]

# Create a dataset with numerical and categorical features
X, _ = make_regression(
    n_samples=1000,
    n_features=3,
    noise=0.1,
    random_state=0,
)
X = pd.DataFrame(X, columns=NUMERICAL_COLS)

def generate_random_string(length):
    return "".join(random.choices(string.ascii_uppercase, k=length))

categorical_feature_length = 2
categorical_features = {}
for c in CATEGORICAL_COLS:
    categorical_column = [generate_random_string(categorical_feature_length) for _ in range(X.shape[0])]
    categorical_features[c] = categorical_column

X = X.assign(**categorical_features)

features_df = session.create_dataframe(X)

# Fit a pipeline with OrdinalEncoder and MinMaxScaler on Snowflake
pipeline = Pipeline(
    steps=[
        (
            "OE",
            OrdinalEncoder(
                input_cols=CATEGORICAL_COLS,
                output_cols=CATEGORICAL_OUTPUT_COLS,
            )
        ),
        (
            "MMS",
            MinMaxScaler(
                input_cols=NUMERICAL_COLS + CATEGORICAL_OUTPUT_COLS,
                output_cols=FEATURE_OUTPUT_COLS,
            )
        ),
    ]
)

pipeline.fit(features_df)

# Use the pipeline to transform a dataset.
result = pipeline.transform(features_df)

In [5]:
import pandas as pd
from sklearn.datasets import make_classification

from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

# Create a session with your preferred method
# session =

FEATURE_COLS = ["X1", "X2", "X3", "X4", "X5", "X6"]
LABEL_COLS = ["Y"]
OUTPUT_COLS = ["PREDICTIONS"]

# Set up data.
X, y = make_classification(
    n_samples=40000,
    n_features=6,
    n_informative=4,
    n_redundant=1,
    random_state=0,
    shuffle=True,
)

X = pd.DataFrame(X, columns=FEATURE_COLS)
y = pd.DataFrame(y, columns=LABEL_COLS)

features_pandas = pd.concat([X, y], axis=1)
features_df = session.create_dataframe(features_pandas)

# Train an XGBoost model on snowflake.
xgboost_model = XGBClassifier(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COLS,
    output_cols=OUTPUT_COLS
)

xgboost_model.fit(features_df)

# Use the model to make predictions.
predictions = xgboost_model.predict(features_df)
predictions[OUTPUT_COLS].show()

RuntimeError: (0000) Package cloudpickle==3.0.0 is not supported in snowflake conda channel for python runtime 3.8.